In [1]:
# import libraries
import pandas as pd
import datetime as dt
import requests
import json

In [2]:
# get start date of us week
today = dt.datetime.today()
week_start = (today - dt.timedelta(days=8)).date()

In [3]:
# get season week for file name later
week = today.strftime('%d%m%Y')

In [4]:
print(week_start)
print(week)

2023-08-29
06092023


***

In [5]:
# get current week affixes

In [6]:
# url for current eu week affixes
affix_url = 'https://raider.io/api/v1/mythic-plus/affixes?region=eu&locale=en'

In [7]:
affix_data = requests.get(affix_url).json()
affixes = affix_data['leaderboard_url'].split('/')[-1]

In [8]:
affixes

'tyrannical-afflicted-bolstering'

***

In [9]:
# get m+ runs urls

In [10]:
# list of available regions
region_list = ['us', 'eu']

In [11]:
# list of 10.1 dungeons
dungeon_list = ['brackenhide-hollow',
                'freehold',
                'halls-of-infusion',
                'neltharions-lair',
                'neltharus',
                'the-underrot',
                'the-vortex-pinnacle',
                'uldaman-legacy-of-tyr']

In [12]:
url_list = []

# iterate over fields and append new urls to list
for region in region_list:
    for dungeon in dungeon_list:
        url_list.append('https://raider.io/api/v1/mythic-plus/runs?season=season-df-2&region=' +region+  \
                                                                                    '&dungeon='+dungeon+ \
                                                                                    '&affixes='+affixes+ \
                                                                                    '&page=')

In [13]:
len(url_list)

16

***

In [14]:
# get data

In [15]:
def get_leaderboard(url):
    runs_list = []
    
    for page in range(100): # iterate over 100 url pages
        page_url = url + str(page) # create full url
        response = requests.get(page_url) # get weekly leaderboard data
    
        # check if page exists
        if response.status_code != 200: break  
        else:
            runs_data = response.json()
    
            for i,j in enumerate(runs_data['rankings']): # iterate over rank (1 rank = 1 dungeon) 
                for k,l in enumerate(runs_data['rankings'][i]['run']['roster']): # iterate over characters in team
                    
                    region = runs_data['rankings'][i]['run']['roster'][k]['character']['region']['slug']
                    keystone_run_id = runs_data['rankings'][i]['run']['keystone_run_id']
                    dungeon_name = runs_data['rankings'][i]['run']['dungeon']['name']
                    mythic_keystone_level = runs_data['rankings'][i]['run']['mythic_level']
                    player_spec = runs_data['rankings'][i]['run']['roster'][k]['character']['spec']['name']
                    player_class = runs_data['rankings'][i]['run']['roster'][k]['character']['class']['name']
                    player_role = runs_data['rankings'][i]['run']['roster'][k]['role']
                    completion_date = runs_data['rankings'][0]['run']['completed_at'][:10]
                    completion_time = runs_data['rankings'][i]['run']['clear_time_ms']
                    affix_1 = runs_data['rankings'][i]['run']['weekly_modifiers'][0]['name']
                    affix_2 = runs_data['rankings'][i]['run']['weekly_modifiers'][1]['name']
                    affix_3 = runs_data['rankings'][i]['run']['weekly_modifiers'][2]['name']
                    
                    # consolidate into 'row' and append to list
                    char_list = [region,
                                 keystone_run_id,
                                 dungeon_name,
                                 mythic_keystone_level,
                                 player_spec,
                                 player_class,
                                 player_role,
                                 completion_date,
                                 completion_time,
                                 affix_1,
                                 affix_2,
                                 affix_3]
                    
                    runs_list.append(char_list)
    
    # create dataframe
    runs_df = pd.DataFrame(runs_list, columns=['region',
                                               'keystone_run_id',
                                               'dungeon_name',
                                               'mythic_keystone_level',
                                               'player_spec',
                                               'player_class',
                                               'player_role',
                                               'completion_date',
                                               'completion_time_ms',
                                               'affix_1',
                                               'affix_2',
                                               'affix_3'])
    
    return runs_df

In [16]:
leaderboard_list = []

# iterate through urls and append new dataframes to list
for idx,url in enumerate(url_list):
    leaderboard_list.append(get_leaderboard(url))
    print(idx)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


In [17]:
# consolidate dataframes
leaderboard_df = pd.concat(leaderboard_list)

***

In [18]:
# simple data cleaning/filtering

In [19]:
# get completion time in minutes
leaderboard_df['completion_time_min'] = round((leaderboard_df['completion_time_ms'] / 60000),3)

In [20]:
# get runs only completed within the week
leaderboard_df['completion_date'] = pd.to_datetime(leaderboard_df['completion_date'], 
                                                   format='%Y-%m-%d', 
                                                   errors='coerce'
                                                  ).dt.date

leaderboard_df = leaderboard_df[leaderboard_df['completion_date']>=week_start]

In [21]:
# get runs only keystone level 25 and above
leaderboard_df = leaderboard_df[leaderboard_df['mythic_keystone_level']>=25]

***

In [22]:
# save data

In [23]:
leaderboard_df.to_excel('data_dfs2/leaderboard_' + week + '.xlsx')